In [ ]:
import os
from osgeo import ogr
import osgeopy as gp

In [ ]:
data_dir = r'osgeopy-data'

## Using spatial references with OSR

In [ ]:
# import osr so you can work with spatial references.
from osgeo import osr

### Spatial reference objects

In [ ]:
# Look at a geographic SRS.
ds = ogr.Open(os.path.join(data_dir, 'us', 'states_48.shp'))
srs = ds.GetLayer().GetSpatialRef()

In [ ]:
# Well Known Text (WKT)
print(srs)

In [ ]:
# PROJ.4
print(srs.ExportToProj4())

In [ ]:
# XML
print(srs.ExportToXML())

In [ ]:
# Look at a UTM SRS.
utm_sr = osr.SpatialReference()
utm_sr.ImportFromEPSG(26912)
print(utm_sr) # WKT
print(utm_sr.ExportToProj4()) # PROJ.4
print(utm_sr.ExportToXML()) # XML

In [ ]:
# Get the projection name.
print(utm_sr.GetAttrValue('PROJCS'))

In [ ]:
# Get the authority.
print(utm_sr.GetAttrValue('AUTHORITY'))
print(utm_sr.GetAttrValue('AUTHORITY', 1))

In [ ]:
# Get the datum code.
print(utm_sr.GetAuthorityCode('DATUM'))

In [ ]:
# Get the false easting.
print(utm_sr.GetProjParm(osr.SRS_PP_FALSE_EASTING))

### Creating spatial reference objects

In [ ]:
# Create a UTM SRS from an EPSG code.
sr = osr.SpatialReference()
sr.ImportFromEPSG(26912)
print(sr.GetAttrValue('PROJCS'))

In [ ]:
# Create a UTM SRS from a PROJ.4 string.
sr = osr.SpatialReference()
sr.ImportFromProj4('''+proj=utm +zone=12 +ellps=GRS80
                      +towgs84=0,0,0,0,0,0,0 +units=m +no_defs ''')
print(sr.GetAttrValue('PROJCS'))

In [ ]:
# Create a unprojected SRS from a WKT string.
wkt = '''GEOGCS["GCS_North_American_1983",
           DATUM["North_American_Datum_1983",
             SPHEROID["GRS_1980",6378137.0,298.257222101]],
           PRIMEM["Greenwich",0.0],
           UNIT["Degree",0.0174532925199433]]'''
sr = osr.SpatialReference(wkt)
print(sr)

In [ ]:
# Create an Albers SRS using parameters.
sr = osr.SpatialReference()
sr.SetProjCS('USGS Albers')
sr.SetWellKnownGeogCS('NAD83')
sr.SetACEA(29.5, 45.5, 23, -96, 0, 0)
#sr.Fixup()
sr.Validate()
print(sr)

### Assigning a SRS to data

In [ ]:
# Make sure that the output folder exists in your data directory before
# trying this example.
out_fn = os.path.join(data_dir, 'output', 'testdata.shp')

# Create an empty shapefile that uses a UTM SRS. If you run this it will
# create the shapefile with a .prj file containing the SRS info.
sr = osr.SpatialReference()
sr.ImportFromEPSG(26912)
ds = ogr.GetDriverByName('ESRI Shapefile').CreateDataSource(out_fn)
lyr = ds.CreateLayer('counties', sr, ogr.wkbPolygon)

### Projecting geometries

In [ ]:
# Get the world landmasses and plot them.
world = gp.get_shp_geom(os.path.join(data_dir, 'global', 'ne_110m_land_1p.shp'))

In [ ]:
# Create a point for the Eiffel Tower.
tower = ogr.Geometry(wkt='POINT (2.294694 48.858093)')
tower.AssignSpatialReference(osr.SpatialReference(osr.SRS_WKT_WGS84_LAT_LONG))

In [ ]:
# Try to reproject the world polygon to Web Mercator. This should spit out
# an error.
web_mercator_sr = osr.SpatialReference()
web_mercator_sr.ImportFromEPSG(3857)
world.TransformTo(web_mercator_sr)

In [ ]:
# Set a config variable and try the projection again.
from osgeo import gdal
gdal.SetConfigOption('OGR_ENABLE_PARTIAL_REPROJECTION', 'TRUE')
web_mercator_sr = osr.SpatialReference()
web_mercator_sr.ImportFromEPSG(3857)
world.TransformTo(web_mercator_sr)
tower.TransformTo(web_mercator_sr)
print(tower)

In [ ]:
# Create a coordinate transformation between Web Mercator and Gall-Peters.
peters_sr = osr.SpatialReference()
peters_sr.ImportFromProj4("""+proj=cea +lon_0=0 +x_0=0 +y_0=0
                             +lat_ts=45 +ellps=WGS84 +datum=WGS84
                             +units=m +no_defs""")
ct = osr.CoordinateTransformation(web_mercator_sr, peters_sr)
world.Transform(ct)

In [ ]:
# Create an unprojected NAD27 SRS and add datum shift info.
sr = osr.SpatialReference()
sr.SetWellKnownGeogCS('NAD27')
sr.SetTOWGS84(-8, 160, 176)
print(sr)